<a href="https://colab.research.google.com/github/LuisIZ/Lab1_AnimalSound_DeepLearning/blob/test_model/notebooks/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 1 - Animal Sound

## Librerías

In [ ]:
# Para redes neuronales
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Para visualización de resultados
import matplotlib.pyplot as plt

# Para procesamiento de audio
import torchaudio
import librosa
import librosa.display

# Para métricas
import sklearn

# Para manipulación de datos
import numpy as np
import pandas as pd

# Otros
import os
from pathlib import Path
import random
import math
from tqdm import tqdm
from google.colab import drive

In [ ]:
# Libreria para decodificar audio en PyTorch tensors
!pip install torchcodec

## GPU

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Seed

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

## Análisis Exploratorio de Datos (EDA)

1. ¿Cuál es el objetivo del laboratorio?

Queremos desarrollar un modelo de clasificación multi-etiqueta que pueda identificar correctamente los animales presentes en las grabaciones de la selva amazónica.

2. ¿Qué tipo de datos tenemos en nuestro dataset?

Tenemos los archivos de audio en formato WAV en las carpetas `train/` y `test/`. Además, en la primera carpeta, tenemos un archivo CSV que tiene como primera columna el nombre del archivo o `filename` y el resto de columnas son los nombres de cada especie. En total son 43 columnas, la primera contendra strings mientras que el resto contendrá valores 0 o 1 que indican la ausencia o presencia de la especie en la grabación.

3. ¿Qué herramientas planeamos utilizar?

En principio, planeamos utilizar `PyTorch` para los modelos (para que utilicen redes neuronales), `Matplotlib` para crear los gráficos de nuestros resultados (ej. modelar el descenso de la gradiente o como va evolucionando los losses en la etapa de training y testing), `TorchAudio` o `Librosa` para analizar features del dataset (ej. Mel-spectogram, MFCC, etc.), `Sklearn` para obtener metricas (ej. f1 score, multilabel confusion matrix, ROC/PR curves etc.) y utilizar métodos de reducción de dimensionalidad (ej. PSA, TSNE, etc.), `Pandas` y `Numpy` para manipular la data y sacar alguna métricas estadísticas (ej. promedio, cuartiles, etc.).

4. ¿Qué restricciones tenemos?

Además del plazo de entrega que es de 1 semana, tenemos recursos computacionales limitados. Trabajaremos con Colab para aprovechar la GPU que nos brinda.

## Google Drive


Verificamos el acceso a Google Drive porque estamos desarrollando el laboratorio en VS Code con la extensión de Google Colab y desde la página web de este último para poder acceder sin problemas a la data en Google Drive que está como acceso directo (*symlink*) a una carpeta compartida.

In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
MYDRIVE = Path("/content/drive/MyDrive")

hits = list(MYDRIVE.rglob("Animal Sounds"))
if not hits:
    raise FileNotFoundError("No encuentro la carpeta 'Animal Sounds' dentro de MyDrive. Revisa que el acceso directo exista.")

DATA_ROOT = hits[0]          # la ruta “atajo”
REAL_ROOT = hits[0].resolve() # la ruta real (como symlink)

print("DATA_ROOT:", DATA_ROOT)
print("REAL_ROOT:", REAL_ROOT)

DATA_ROOT: /content/drive/MyDrive/Animal Sounds
REAL_ROOT: /content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds


In [ ]:
TRAIN_7Z  = REAL_ROOT / "train.7z"
TEST_7Z   = REAL_ROOT / "test.7z"
TRAIN_CSV = REAL_ROOT / "train.csv"

for p in [TRAIN_7Z, TEST_7Z, TRAIN_CSV]:
    print(p, "=>", p.exists())

/content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds/train.7z => True
/content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds/test.7z => True
/content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds/train.csv => True


Habiendo encontrado las rutas de nuestros archivos:

- La carpeta comprimida con los datos de training (`train.7z`)
- La carpeta comprimida con los datos de testing (`test.7z`)
- El archivo csv con las multi-etiquetas de cada video, indicando que especie suena en el audio, 1, y cual no, 0 (`train.csv`)

En la siguiente sección, procedemos a descomprimir las carpetas, utilizando la herramienta `7z`, y revisar su contenido para poder realizar el Análisis Exploratorio de Datos (o *EDA* por sus siglas en Inglés).

## Dataset

Como trabajar con los datos directamente en Drive sería muy lento, procederemos a guardar el contenido de los archivos descomprimidos en `/content` que es el disco local temporal de la máquina de Colab que ofrece mayor velocidad de I/O. Al ser temporal, si se reinicia el entorno, es necesario repetir la descompresión.

In [ ]:
# Verificamos que haya suficiente espacio en /content
!df -h /content

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   44G   65G  40% /


In [ ]:
# Definimos rutas y creamos carpetas para guardar los datos
DRIVE_DATA = Path("/content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds")
TRAIN_7Z = DRIVE_DATA / "train.7z"
TEST_7Z  = DRIVE_DATA / "test.7z"
TRAIN_CSV = DRIVE_DATA / "train.csv"

OUT_BASE = Path("/content/data")
TRAIN_OUT = OUT_BASE / "train"
TEST_OUT  = OUT_BASE / "test"

TRAIN_OUT.mkdir(parents=True, exist_ok=True)
TEST_OUT.mkdir(parents=True, exist_ok=True)

print(TRAIN_7Z.exists(), TEST_7Z.exists(), TRAIN_CSV.exists())
print("Extracción a:", OUT_BASE)

True True True
Extracción a: /content/data


In [ ]:
# Extraemos los datos
!7z x "/content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds/train.7z" -o"/content/data/train" -y
!7z x "/content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds/test.7z"  -o"/content/data/test"  -y


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/.shortcut-targe . wXtSAsIL9HQJ3yf/Animal Sounds/                                                                         1 file, 7069498436 bytes (6742 MiB)

Extracting archive: /content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds/train.7z
--
Path = /content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds/train.7z
Type = 7z
Physical Size = 7069498436
Headers Size = 727932
Method = LZMA2:23
Solid = +
Blocks = 1

  0%      0% - train/INCT17_20191113_040000_0_3.wav                 

In [ ]:
# Verificamos si estan los audios
!find /content/data/train -maxdepth 2 -type f | head
!find /content/data/test  -maxdepth 2 -type f | head

/content/data/train/train/INCT17_20191113_040000_0_3.wav
/content/data/train/train/INCT17_20191113_040000_10_13.wav
/content/data/train/train/INCT17_20191113_040000_11_14.wav
/content/data/train/train/INCT17_20191113_040000_12_15.wav
/content/data/train/train/INCT17_20191113_040000_13_16.wav
/content/data/train/train/INCT17_20191113_040000_14_17.wav
/content/data/train/train/INCT17_20191113_040000_15_18.wav
/content/data/train/train/INCT17_20191113_040000_16_19.wav
/content/data/train/train/INCT17_20191113_040000_17_20.wav
/content/data/train/train/INCT17_20191113_040000_18_21.wav
/content/data/test/test/INCT17_20191125_040000_0_3.wav
/content/data/test/test/INCT17_20191125_040000_10_13.wav
/content/data/test/test/INCT17_20191125_040000_11_14.wav
/content/data/test/test/INCT17_20191125_040000_12_15.wav
/content/data/test/test/INCT17_20191125_040000_13_16.wav
/content/data/test/test/INCT17_20191125_040000_14_17.wav
/content/data/test/test/INCT17_20191125_040000_15_18.wav
/content/data/t

In [ ]:
DATA_DIR = Path("/content/data")

def resolve_nested(split_dir: Path):
    nested = split_dir / split_dir.name
    return nested if nested.exists() else split_dir

TRAIN_DIR = resolve_nested(DATA_DIR / "train")
TEST_DIR  = resolve_nested(DATA_DIR / "test")

print("TRAIN_DIR:", TRAIN_DIR)
print("TEST_DIR :", TEST_DIR)
print("Ejemplo train existe:", TRAIN_DIR.exists())
print("Ejemplo test existe :", TEST_DIR.exists())

TRAIN_DIR: /content/data/train/train
TEST_DIR : /content/data/test/test
Ejemplo train existe: True
Ejemplo test existe : True


In [ ]:
df = pd.read_csv(TRAIN_CSV)
print(df.shape)
df.head()

(62191, 43)


,filename,SPHSUR,BOABIS,SCIPER,DENNAH,LEPLAT,RHIICT,BOALEP,BOAFAB,PHYCUV,...,SCINAS,LEPNOT,ADEMAR,BOAALM,PHYDIS,RHIORN,LEPFLA,SCIRIZ,DENELE,SCIALT
0,INCT20955_20190909_050000_0_3.wav,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,INCT20955_20190909_050000_1_4.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,INCT20955_20190909_050000_2_5.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,INCT20955_20190909_050000_3_6.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,INCT20955_20190909_050000_4_7.wav,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
f0 = df["filename"].iloc[0]
print("Archivo:", f0)
print("Existe?:", (TRAIN_DIR / f0).exists())

Archivo: INCT20955_20190909_050000_0_3.wav
Existe?: True


In [ ]:
drive_root = Path("/content/drive")

train7z = list(drive_root.rglob("train.7z"))
test7z  = list(drive_root.rglob("test.7z"))
csvs    = list(drive_root.rglob("train.csv"))

print("train.7z encontrados:", len(train7z))
print("test.7z encontrados:", len(test7z))
print("train.csv encontrados:", len(csvs))

# candidato: carpeta que tenga los 3 archivos
candidates = []
for p in train7z:
    folder = p.parent
    if (folder / "test.7z").exists() and (folder / "train.csv").exists():
        candidates.append(folder)

print("Carpetas candidatas:", len(candidates))
for c in candidates[:10]:
    print(" -", c)

DATASET_DIR = candidates[0]  # si sale >1, elegimos la que corresponde
print("Usando DATASET_DIR:", DATASET_DIR)

train.7z encontrados: 1
test.7z encontrados: 1
train.csv encontrados: 1
Carpetas candidatas: 1
 - /content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds
Usando DATASET_DIR: /content/drive/.shortcut-targets-by-id/1F5_zs2zy0oECJu6NSwXtSAsIL9HQJ3yf/Animal Sounds


## EDA

### Labels

In [ ]:
label_cols = df.columns[1:]   # las columnas de todas las especies en un audio
Y = df[label_cols].astype(int)

k = Y.sum(axis=1)
print("Shape:", df.shape)
print("Num audios:", len(df))
print("Num clases:", len(label_cols))
print("Audios únicos:", df["filename"].nunique())
print("Nulos totales:", df.isna().sum().sum())
print("Porcentaje audios sin etiquetas:", (k==0).mean()*100)

Shape: (62191, 43)
Num clases: 42
Audios únicos: 62191
Nulos totales: 0


In [ ]:
plt.figure()
plt.hist(k, bins=np.arange(k.max()+2)-0.5)
plt.xlabel("# etiquetas por audio")
plt.ylabel("conteo")
plt.title("Distribución de cardinalidad multi-label")
plt.show()

In [ ]:
# ¿cuántas etiquetas por audio?
k = df[label_cols].sum(axis=1)
k.describe()

,0
count,62191.000000
mean,1.509463
std,1.539720
min,0.000000
25%,0.000000
50%,1.000000
75%,3.000000
max,8.000000


In [ ]:
# balance por clase
class_counts = Y.sum().sort_values(ascending=False)
print("Top 10 clases:\n", class_counts.head(10))
print("\nBottom 10 clases:\n", class_counts.tail(10))

(SPHSUR    13258
 BOABIS    10888
 BOAFAB     6438
 DENMIN     6070
 LEPPOD     6032
 PHYALB     5374
 LEPLAT     5244
 PITAZU     4873
 PHYCUV     4240
 DENNAN     3801
 dtype: int64,
 PHYMAR    200
 DENELE    149
 SCIRIZ     73
 AMEPIC     68
 LEPELE     34
 RHIORN     21
 RHISCI     11
 LEPFLA      7
 SCIFUS      0
 SCINAS      0
 dtype: int64)

In [ ]:
plt.figure()
plt.plot(class_counts.values)
plt.yscale("log")
plt.xlabel("clase (ordenada por frecuencia)")
plt.ylabel("conteo (log)")
plt.title("Imbalance por clase (escala log)")
plt.show()

In [ ]:
# porcentaje de precensia por clase
class_pct = (class_counts / len(df) * 100).sort_values(ascending=False)
class_pct.head(10)

,0
SPHSUR,21.318197
BOABIS,17.507356
BOAFAB,10.351980
DENMIN,9.760255
LEPPOD,9.699153
PHYALB,8.641122
LEPLAT,8.432088
PITAZU,7.835539
PHYCUV,6.817707
DENNAN,6.111817


In [ ]:
# clases con 0 o muy pocas apariciones
zero_classes = class_counts[class_counts==0].index.tolist()
rare_classes = class_counts[class_counts<100].index.tolist()
print("Clases con 0 en train:", zero_classes)
print("Clases con <100 en train:", len(rare_classes))

### Co-ocurrencias

In [ ]:
# matriz co-ocurrencia (42x42)
cooc = (Y.T @ Y).astype(int)
np.fill_diagonal(cooc.values, 0)

# top pares
pairs = []
for i, a in enumerate(label_cols):
    for j, b in enumerate(label_cols):
        if j <= i:
            continue
        c = cooc.loc[a, b]
        if c > 0:
            pairs.append((c, a, b))
pairs.sort(reverse=True)

print("Top 15 pares más comunes:")
for c,a,b in pairs[:15]:
    print(f"{a} + {b}: {c}")

### Audio

In [ ]:
def audio_info_fast(path):
    waveform, sample_rate = torchaudio.load(str(path))
    num_channels = waveform.shape[0]
    num_frames = waveform.shape[1]
    duration = num_frames / sample_rate
    return sample_rate, num_frames, duration, num_channels

In [ ]:
sample_n = 1000
sample_files = df["filename"].sample(sample_n, random_state=SEED).tolist()

In [ ]:
srs, durs, chs = [], [], []
for fn in tqdm(sample_files):
    p = TRAIN_DIR / fn
    sr, nframes, dur, nch = audio_info_fast(p)
    srs.append(sr); durs.append(dur); chs.append(nch)

In [ ]:
print("Sample rate (valores únicos aprox):", sorted(set(srs))[:10], "...")
print("Canales (valores únicos):", sorted(set(chs)))
print("Duración promedio:", np.mean(durs), "sec")
print("Duración min/max:", np.min(durs), np.max(durs))

In [ ]:
plt.figure()
plt.hist(durs, bins=30)
plt.xlabel("duración (s)")
plt.ylabel("conteo")
plt.title("Distribución de duración (muestra)")
plt.show()

In [ ]:
def plot_wave_and_melspec(wav_path, target_sr=22050):
    y, sr = librosa.load(wav_path, sr=target_sr, mono=True)
    plt.figure()
    plt.plot(y)
    plt.title(f"Waveform | sr={sr} | len={len(y)/sr:.2f}s")
    plt.show()

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, n_fft=1024, hop_length=512)
    S_db = librosa.power_to_db(S, ref=np.max)
    plt.figure()
    librosa.display.specshow(S_db, sr=sr, hop_length=512, x_axis='time', y_axis='mel')
    plt.colorbar(format="%+2.0f dB")
    plt.title("Log-Mel Spectrogram")
    plt.show()

In [ ]:
# ejemplo: uno random
fn = df["filename"].iloc[0]
plot_wave_and_melspec(str(TRAIN_DIR/fn))

## Feature Extraction

In [ ]:
# creamos carpeta para almacenar features
CACHE_DIR = Path("/content/drive/MyDrive/Lab1_cache")
CACHE_DIR.mkdir(parents=True, exist_ok=True)
print(CACHE_DIR)

/content/drive/MyDrive/Lab1_cache


In [ ]:
def extract_mfcc_stats(wav_path, sr=32000, n_mfcc=20):
    y, _ = librosa.load(wav_path, sr=sr, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    feat = np.concatenate([mfcc.mean(axis=1), mfcc.std(axis=1)]).astype(np.float32)
    return feat

In [ ]:
n_mfcc = 20
feat_names = [f"mfcc_mean_{i}" for i in range(n_mfcc)] + [f"mfcc_std_{i}" for i in range(n_mfcc)]

In [ ]:
# OJO: Solo correrlo si necesitas volver a extraer y guardar los features
if H5_PATH.exists():
    os.remove(H5_PATH)

In [ ]:
# =========================
# CONFIG
# =========================
CACHE_DIR = Path("/content/drive/MyDrive/Lab1_cache")
CACHE_DIR.mkdir(parents=True, exist_ok=True)

H5_PATH = CACHE_DIR / "train_mfcc_stats.h5"
KEY = "train"
BATCH = 256  # ¿probar 256 o 512?

# para evitar el error de "string len limit"
MIN_ITEMSIZE = {"filename": 120}

# =========================
# (OPCIONAL) LIMPIAR H5 SI ESTÁ "MAL CREADO"
# =========================
# Si sale el error de itemsize, lo más limpio es borrar y regenerar
if H5_PATH.exists():
    print("H5 ya existe:", H5_PATH)
    print("Si este archivo fue creado antes SIN min_itemsize, podría fallar. "
          "Si vuelve a fallar, bórralo (os.remove) y reintenta.")
    # Descomentar si se quiere forzar recreación
    # os.remove(H5_PATH)

# =========================
# RECUPERAR PROCESADOS
# =========================
processed = set()
if H5_PATH.exists():
    try:
        with pd.HDFStore(H5_PATH, mode="r") as store:
            if f"/{KEY}" in store.keys():
                processed = set(store.select(KEY, columns=["filename"])["filename"].astype(str).tolist())
    except Exception as e:
        print("No se pudo leer el H5 para resume. Error:", repr(e))
        print("Recomendación: borrar el H5 y regenerar.")
        # os.remove(H5_PATH)
        processed = set()

print("Procesados previamente:", len(processed))

# =========================
# EXTRACCIÓN + GUARDADO POR BATCH
# =========================
rows = []
skipped_missing = 0
skipped_errors = 0
written = 0

for i in tqdm(range(len(df)), desc="Extrayendo MFCC"):
    fn = str(df.loc[i, "filename"])

    if fn in processed:
        continue

    wav_path = TRAIN_DIR / fn
    if not wav_path.exists():
        skipped_missing += 1
        continue

    try:
        feat = extract_mfcc_stats(str(wav_path), sr=32000, n_mfcc=n_mfcc)
    except Exception:
        skipped_errors += 1
        continue

    row = {"filename": fn}
    row.update({k: float(v) for k, v in zip(feat_names, feat)})

    for c in label_cols:
        row[c] = int(df.loc[i, c])

    rows.append(row)

    # flush por batch
    if len(rows) >= BATCH:
        out = pd.DataFrame(rows)

        # Guardar (con min_itemsize para filename)
        out.to_hdf(
            H5_PATH,
            key=KEY,
            mode="a",
            format="table",
            append=True,
            data_columns=["filename"],
            min_itemsize=MIN_ITEMSIZE,
            complib="blosc",
            complevel=5
        )

        written += len(out)
        rows = []

# flush final
if rows:
    out = pd.DataFrame(rows)
    out.to_hdf(
        H5_PATH,
        key=KEY,
        mode="a",
        format="table",
        append=True,
        data_columns=["filename"],
        min_itemsize=MIN_ITEMSIZE,
        complib="blosc",
        complevel=5
    )
    written += len(out)

print("Listo. Guardado en:", H5_PATH)
print("Escritos nuevos:", written)
print("Saltados (faltan archivos):", skipped_missing)
print("Saltados (errores lectura/audio):", skipped_errors)

Procesados previamente: 0


Extrayendo MFCC: 100%|██████████| 62191/62191 [23:31<00:00, 44.05it/s]


Listo. Guardado en: /content/drive/MyDrive/Lab1_cache/train_mfcc_stats.h5
Escritos nuevos: 62191
Saltados (faltan archivos): 0
Saltados (errores lectura/audio): 0


In [ ]:
SAMPLE_N = 300
sample_files = df["filename"].sample(SAMPLE_N, random_state=42).tolist()
wav_paths = [str(TRAIN_DIR / f) for f in sample_files]

In [ ]:
H5_PATH = "/content/drive/MyDrive/Lab1_cache/train_mfcc_stats.h5"
df_feat = pd.read_hdf(H5_PATH, key="train")

print("Shape:", df_feat.shape)
print("Cols:", df_feat.columns[:10].tolist())
print("Filenames únicos:", df_feat["filename"].nunique())
print("Nulos totales:", df_feat.isna().sum().sum())

df_feat.head()

Shape: (62191, 83)
Cols: ['filename', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8']
Filenames únicos: 62191
Nulos totales: 0


,filename,mfcc_mean_0,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,mfcc_mean_7,mfcc_mean_8,...,SCINAS,LEPNOT,ADEMAR,BOAALM,PHYDIS,RHIORN,LEPFLA,SCIRIZ,DENELE,SCIALT
0,INCT20955_20190909_050000_0_3.wav,-189.088409,117.389236,-146.154938,55.875923,-7.179368,52.681614,-11.384820,22.849953,0.413092,...,0,0,0,0,0,0,0,0,0,0
1,INCT20955_20190909_050000_1_4.wav,-187.898651,116.883789,-146.542892,56.535400,-7.780136,51.700756,-9.963317,22.915176,-0.442919,...,0,0,0,0,0,0,0,0,0,0
2,INCT20955_20190909_050000_2_5.wav,-187.243317,109.707817,-139.818100,50.498848,-2.099809,47.686493,-6.434562,21.802526,0.812162,...,0,0,0,0,0,0,0,0,0,0
3,INCT20955_20190909_050000_3_6.wav,-185.078537,116.537796,-149.052811,55.316799,-7.776229,52.924919,-10.019419,23.378138,0.383413,...,0,0,0,0,0,0,0,0,0,0
4,INCT20955_20190909_050000_4_7.wav,-175.066849,107.470642,-139.312286,47.302567,-0.816672,48.910305,-7.007190,22.232889,2.580527,...,0,0,0,0,0,0,0,0,0,0
